In [1]:
import glob
import re

INPUT_FOLDER = '../../../travelogues-corpus/16th_century/books'

def read_file(f):
  with open(f, 'r') as file:
    str = file.read()
    ascii_only = re.sub('[^A-Za-z0-9 ]+', '', str)
    return re.sub('\\s+', ' ', ascii_only)

filenames = [f for f in glob.glob(INPUT_FOLDER + '**/*.txt')]
texts = [ read_file(f) for f in filenames ]

f'Loaded {len(texts)} texts'

'Loaded 66 texts'

In [2]:
extra_stopwords = []

with open('stopwords.txt') as f:
  extra_stopwords = f.readlines()
  extra_stopwords = [ x.strip() for x in extra_stopwords ]

In [3]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('german')
stop_words.extend(extra_stopwords)
stop_words.sort()

[nltk_data] Downloading package stopwords to /home/simonr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import itertools
import matplotlib.pyplot as plt
import string

# We'll do stemming to avoid duplicate variants
porter = PorterStemmer()

def stem_message(text):
  words = word_tokenize(text)

  stemmed_words = []
  for word in words:
    stemmed_words.append(porter.stem(word))
    
  # Remove stopwords here
  clean_tokens = [w for w in stemmed_words if not w in stop_words and len(w) > 3] 
  return ' '.join(clean_tokens)

cleaned_texts = []
for text in texts:
  # Remove punctutation
  text = text.translate(str.maketrans('', '', string.punctuation))
  cleaned_texts.append(stem_message(text))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')
tf = vectorizer.fit_transform(cleaned_texts).toarray()

tf_feature_names = vectorizer.get_feature_names()

number_of_topics = 6
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)
model.fit(tf)

/home/simonr/Software/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=6, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [10]:
import pandas as pd

def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 20
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights
0,schiff,1607.5,berg,1256.7,feind,493.5,priester,525.6,dafi,230.0,trcken,1512.0
1,insel,697.7,meer,1061.3,groen,439.5,kirchen,517.4,glauben,214.9,christen,620.1
2,wilden,696.7,heiligen,878.0,jnen,421.1,johann,507.6,christen,201.2,trckischen,560.7
3,hatt,649.8,herr,850.4,trcken,373.4,krieg,467.8,feind,160.9,krieg,502.0
4,meer,644.5,christen,735.0,groe,305.5,meer,433.4,volck,156.0,namen,501.5
5,dritt,453.6,weit,714.8,herrn,298.8,knigreich,409.4,leben,145.0,feind,479.4
6,were,434.7,herrn,628.3,spanier,274.5,herr,386.3,dieweil,139.9,sohn,428.5
7,waffer,423.3,schiff,626.5,dieweil,273.9,feind,383.8,groffen,127.1,keyser,424.0
8,indien,412.5,venedig,610.4,namen,266.1,namen,375.8,mgen,127.0,trckisch,384.7
9,schiffen,393.9,stadt,603.0,meer,254.3,bald,375.7,jhnen,123.6,dafi,363.9
